# Personal Finance Analysis - Money v1 MVP

Secure analysis of Monzo transactions stored locally.

**Security Notes:**
- Data is stored locally in `/data/transactions.csv`
- No sensitive API tokens are stored or displayed
- Run this notebook in a secure environment

**Goals:**
- Display recent transactions
- Calculate monthly spending totals by category
- Prepare for future interactive dashboards

In [ ]:
# Import required libraries
import pandas as pd
import os
from datetime import datetime
import matplotlib.pyplot as plt  # For future charts

# Set pandas display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

: 

In [ ]:
# Read transactions from local CSV
csv_path = '/data/transactions.csv'

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print(f"Loaded {len(df)} transactions from {csv_path}")

    # Convert date columns to datetime
    if 'created' in df.columns:
        df['created'] = pd.to_datetime(df['created'])
    if 'fetched_at' in df.columns:
        df['fetched_at'] = pd.to_datetime(df['fetched_at'])

    print("Data preview:")
    print(df.head())
else:
    print(f"CSV file not found at {csv_path}")
    print("Run the fetch script first: python scripts/monzo_fetch.py")
    df = pd.DataFrame()  # Empty dataframe

In [ ]:
# Display last 5 transactions
if not df.empty:
    print("\n=== Last 5 Transactions ===")
    # Sort by created date descending
    recent_transactions = df.sort_values('created', ascending=False).head(5)

    # Display relevant columns
    display_cols = ['created', 'description', 'amount', 'category']
    available_cols = [col for col in display_cols if col in df.columns]

    print(recent_transactions[available_cols])
else:
    print("No transactions to display")

In [ ]:
# Calculate monthly totals per category
if not df.empty and 'created' in df.columns and 'amount' in df.columns and 'category' in df.columns:
    print("\n=== Monthly Totals by Category ===")

    # Convert amount to numeric (Monzo amounts are in pence)
    df['amount_numeric'] = pd.to_numeric(df['amount'], errors='coerce') / 100  # Convert to pounds

    # Add month-year column
    df['month_year'] = df['created'].dt.to_period('M')

    # Group by month and category, sum amounts
    monthly_totals = df.groupby(['month_year', 'category'])['amount_numeric'].sum().unstack().fillna(0)

    print("Monthly spending totals (£):")
    print(monthly_totals)

    # Summary for current month
    current_month = datetime.now().strftime('%Y-%m')
    if current_month in [str(period) for period in monthly_totals.index]:
        print(f"\n=== Current Month ({current_month}) Summary ===")
        current_data = monthly_totals.loc[current_month]
        print(current_data)
        print(f"Total spending: £{current_data.sum():.2f}")
    else:
        print(f"\nNo data for current month ({current_month}) yet")

else:
    print("Missing required columns for monthly analysis")

## Future Dashboard Integration

**Placeholder for Interactive Charts:**

This section will be expanded to include:
- Monthly spending trends over time
- Category breakdown pie charts
- Budget vs actual comparisons
- Interactive filters and date ranges

**Integration with Personal Site:**
- Export summary data to JSON for Hugo data templates
- Create dedicated `/money/` section on the site
- Add charts using Chart.js or D3.js in Hugo templates
- Ensure WCAG AA accessibility for all visualizations

**ML Enhancements (Future):**
- Smart categorization using NLP
- Anomaly detection for unusual spending
- Predictive budgeting based on historical data
- Personalized financial insights

## Automation Instructions

### Manual Execution
```bash
# Set environment variable (replace with your actual token)
export MONZO_ACCESS_TOKEN=your_token_here

# Run the fetch script
python scripts/monzo_fetch.py
```

### Daily Automation (macOS/Linux)
1. Create a cron job:
   ```bash
   crontab -e
   ```
2. Add this line to run daily at 6 AM:
   ```
   0 6 * * * cd /path/to/your/site && MONZO_ACCESS_TOKEN=your_token python scripts/monzo_fetch.py
   ```

### Daily Automation (Windows)
1. Create a batch file `fetch_transactions.bat`:
   ```
   @echo off
   set MONZO_ACCESS_TOKEN=your_token_here
   python scripts\monzo_fetch.py
   ```
2. Use Task Scheduler to run daily

### GitHub Actions (for automated deployment)
Add a workflow that runs the script and commits updated data (but never commit tokens!)

### Security Best Practices
- Store `MONZO_ACCESS_TOKEN` securely (never in code)
- Use `.env` files locally with `python-dotenv`
- Rotate tokens regularly
- Monitor API usage to avoid rate limits
- Keep data local; never expose in web interfaces without authentication

### Extending to Site Dashboards
1. Add `/data/summaries.json` generation to the script
2. Create Hugo data templates in `/data/`
3. Add `/money/` section to site navigation
4. Use Chart.js for accessible visualizations
5. Ensure all charts meet WCAG AA contrast and keyboard navigation requirements